



# Gaussian process classification

In the case of binomial or multinomial random variables the likelihood is obviously non-Gaussian. Thus the posterior $p(\mathbf{f} \mid \mathbf{X}, \mathbf{y})$ also is non-Gaussian,  has no analytical form and has to be approximated. A simple, straightforward approximation is Laplace's method using Newton updates:
\begin{align}
P(\mathbf{f} \mid \mathbf{x}, \mathbf{y}) & \approx q(\mathbf{f} \mid \mathbf{x}, \mathbf{y}) \\
& = \mathcal{N}( \bar{\mathbf{f}} , (  \mathbf{K}^{-1} + \mathbf{W}  )^{-1}  ),
\end{align}
where $\mathbf{W} =  - \nabla \nabla \log (\mathbf{y} \mid \mathbf{f})$. For instance, in a binomial setting using the logistic function
\begin{equation}
\mathbf{W} = \operatorname{diag}(\sigma_1(1 - \sigma_1), \dots,  \sigma_n(1 - \sigma_n))
\end{equation}
\cite{rasmussen2006gaussian, barberBRML2012} shows the details.  Using the approximation of the posterior, we can calculate the predictive distribution of a latent variable $f^*$:
\begin{align}
q(f^* \mid x^*, \mathbf{x}, \mathbf{y}) & = \int P(f^* \mid {x}^*, \mathbf{x}, \mathbf{f}) q(\mathbf{f} \mid \mathbf{x}, \mathbf{y})d\mathbf{f} \\
\mathbb{E}[f^* \mid x^*, \mathbf{x}, \mathbf{y}] & =  \kappa({x}^*, \mathbf{x} )(\mathbf{y} -  \sigma(\bar{\mathbf{f}})) \\
\mathbb{V}[f^* \mid x^*, \mathbf{x}, \mathbf{y}] & =  \kappa({x}^*, {x}^*) - \kappa({x}^*, \mathbf{x} ) (\mathbf{K} + \mathbf{W}^{-1} )^{-1}  \kappa(\mathbf{x}, {x}^*)
\end{align}

The class of a novel data-point $y^*$ is then calculated as:
\begin{align}
P(y^* = 1 \mid x^*, \mathbf{x}, \mathbf{y}) & = \int \sigma(y^*) q(f^* \mid x^*, \mathbf{x}, \mathbf{y}) d f^* \\
& =   \int \sigma(y^*) \mathcal{N}(\mathbb{E}[f^*], \mathbb{V}[f^*]  ) d f^*,
\end{align}
which can be calculated using the gaussian error function \cite{lnorm}.
